# ISM6251.003S23- Assignment 2


## Business Problem
The aim is to create a predictive model using the patient characteristics provided, which can determine if a patient has heart disease or not. If the model proves effective, it could be utilized by the medical sector to anticipate heart disease in new patients who possess the necessary data. This will enable them to prioritize patients for diagnostic testing, surgeries, and other treatments.

## Description of the analysis
This project aims to determine the evaluation metrics employed for assessing the performance of the model(s) and explain the rationale behind the selection of these metrics. The logistic regression, SVM, and decision tree model, MLP and Keras technique will be utilized for modeling purposes. To test a variety of parameter values for each model, both random and grid searches will be performed for each of the model.

### 1. Library Import

In [2]:
import numpy as np
import pandas as pd
import warnings
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier 
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report
import tensorflow as tf
from tensorflow import keras
from keras.initializers import GlorotNormal

### 2. Load the data

In [3]:
X_train = pd.read_csv("heart_train_X.csv")
X_test = pd.read_csv("heart_test_X.csv")
y_train = pd.read_csv("heart_train_y.csv")
y_test = pd.read_csv("heart_test_y.csv")


### 2.1 Model the data

Dataframe will be constructed to contain all the outcomes obtained from our models

In [4]:
performance = pd.DataFrame({"model": [], "Accuracy": [], "Precision": [], "Recall": [], "F1": []})

### 3. Choosing the right performance metric

In this dataset, a false-negative result would indicate that a person with heart disease is wrongly classified as healthy. Therefore, it is crucial to choose a model that minimizes the number of false negatives and thus, maximizes the recall value. Also, the cost of a false-negative result in this model is very high and would result in death of the patient.

Hence, recall will be used to measure the performance of the models. 

### 4. Logistic Regression 

#### 4.1 Define parameter distribution

In [5]:
param_grid_lr = {'penalty': ['None','l1', 'l2']}

#### 4.2 Performing Random Search

In [7]:
#Defining scoring metric
score_measure = "recall"

# define the logistic regression model
model = LogisticRegression()

# Create a random search object
random_search_lr = RandomizedSearchCV(estimator=model, param_distributions=param_grid_lr, n_iter=100, cv=5, random_state=42)

# Fit the random search to the data
_ = random_search_lr.fit(X_train, y_train)

# Print the best parameters found
print(f"The best {score_measure} score is {random_search_lr.best_score_}")
print(f"... with parameters: {random_search_lr.best_params_}")

warnings.filterwarnings('ignore')

The best recall score is 0.8243968739381582
... with parameters: {'penalty': 'l2'}


#### 4.3 Grid Search

In [8]:
#Defining scoring metric
score_measure = "recall"
kfolds = 5

#Using best parameters found from random search
param_grid_lrg = {'penalty': ['l2']}

# define the logistic regression model
model = LogisticRegression()

# define the grid search
grid_search_lr = GridSearchCV(estimator = model, param_grid=param_grid_lrg, cv=kfolds, 
                           scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)

# fit the grid search to the data
_ = grid_search_lr.fit(X_train, y_train)


#Best parameters
print(f"Best parameters: {grid_search_lr.best_params_}")



Fitting 5 folds for each of 1 candidates, totalling 5 fits
Best parameters: {'penalty': 'l2'}


#### 4.4 Predict with test data

In [9]:
c_matrix = confusion_matrix(y_test, grid_search_lr.predict(X_test))
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]

performance = pd.concat([performance, pd.DataFrame({'model':"Logistic Regression", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,Logistic Regression,0.772532,0.803571,0.743802,0.772532


### 5. SVM

#### 5.1 Define parameter distribution

In [10]:
param_grid_SVM = [
  {'C': [1, 10, 100, 1000], 'kernel': ['linear']},
  {'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001], 'kernel': ['rbf']},
 ]

#### 5.2  Performing Random Search

In [11]:
# Create an SVM classifier
svm = SVC()

# Create a random search object
random_search_SVM = RandomizedSearchCV(
    svm, 
    param_distributions=param_grid_SVM, 
    n_iter=50, # number of parameter combinations to try
    cv=5, # number of cross-validation folds
    scoring='recall', # scoring metric to use
    verbose=1, 
    n_jobs=-1 # number of CPU cores to use for parallel computation
)


# Fit the random search to the data
_= random_search_SVM.fit(X_train, y_train)

# Print the best parameters found
print("Best parameters found:", random_search_SVM.best_params_)


Fitting 5 folds for each of 12 candidates, totalling 60 fits
Best parameters found: {'kernel': 'rbf', 'gamma': 0.0001, 'C': 1}


#### 5.3 Grid Search

In [12]:
#Using the best parameters found from random search
param_grid_SVMg = [
  {'C': [1], 
   'kernel': ['rbf'],
   'gamma': [0.0001]}
  
]

# Create an SVM classifier
svm = SVC()

# Create a grid search object
grid_search_SVM = GridSearchCV(
    svm, 
    param_grid=param_grid_SVMg, 
    cv=5, # number of cross-validation folds
    scoring='recall', # scoring metric to use
    verbose=1, 
    n_jobs=-1 # number of CPU cores to use for parallel computation
)

# Fit the grid search to the data
grid_search_SVM.fit(X_train, y_train)



Fitting 5 folds for each of 1 candidates, totalling 5 fits


GridSearchCV(cv=5, estimator=SVC(), n_jobs=-1,
             param_grid=[{'C': [1], 'gamma': [0.0001], 'kernel': ['rbf']}],
             scoring='recall', verbose=1)

In [13]:
c_matrix = confusion_matrix(y_test, grid_search_SVM.predict(X_test))
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]

performance = pd.concat([performance, pd.DataFrame({'model':"SVM", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,Logistic Regression,0.772532,0.803571,0.743802,0.772532
0,SVM,0.519313,0.519313,1.000000,0.683616


### 6. Decision Tree

#### 6.1 Define Parameter Distribution

In [14]:
param_grid_dtree = {
    'min_samples_split': np.arange(1,200),  
    'min_samples_leaf': np.arange(1,200),
    'max_leaf_nodes': np.arange(5, 200), 
    'max_depth': np.arange(1,50), 
    'criterion': ['entropy', 'gini'],
}

#### 6.2 Perform Random Search

In [15]:
# Define scoring measure
score_measure = "recall"
kfolds = 5

# Create a decision tree object
dtree = DecisionTreeClassifier()

# Use RandomizedSearchCV to search over the parameter grid
random_search_dtree = RandomizedSearchCV(estimator = dtree, param_distributions=param_grid_dtree, cv=kfolds, n_iter=500,
                           scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)

# Fit the random search object to the data
_ = random_search_dtree.fit(X_train, y_train)

# Print the best parameters and score

print(f"The best {score_measure} score is {random_search_dtree.best_score_}")
print(f"... with parameters: {random_search_dtree.best_params_}")


warnings.filterwarnings('ignore')

Fitting 5 folds for each of 500 candidates, totalling 2500 fits
The best recall score is 0.8442857142857143
... with parameters: {'min_samples_split': 161, 'min_samples_leaf': 27, 'max_leaf_nodes': 179, 'max_depth': 12, 'criterion': 'gini'}


#### 6.3 Grid Search

In [16]:
#Define scoring measure
score_measure = "recall"
kfolds = 5

#Using best parameters found from random search
param_grid_dtreeg = {
    'min_samples_split': np.arange(176,180),  
    'min_samples_leaf': np.arange(22,26),
    'max_leaf_nodes': np.arange(49,53), 
    'max_depth': np.arange(20,24), 
    'criterion': ['gini'],
}

# Create a decision tree object
dtree = DecisionTreeClassifier()

# Create a grid search object
grid_search_dtree = GridSearchCV(estimator = dtree, param_grid=param_grid_dtreeg, cv=kfolds, 
                           scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)

# Fit the grid search to the data
_ = grid_search_dtree.fit(X_train, y_train)


# Print the best parameters and score
print(f"The best {score_measure} score is {grid_search_dtree.best_score_}")
print(f"... with parameters: {grid_search_dtree.best_params_}")


Fitting 5 folds for each of 256 candidates, totalling 1280 fits
The best recall score is 0.8442857142857143
... with parameters: {'criterion': 'gini', 'max_depth': 20, 'max_leaf_nodes': 49, 'min_samples_leaf': 23, 'min_samples_split': 176}


In [17]:
c_matrix = confusion_matrix(y_test, grid_search_dtree.predict(X_test))
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"Decision tree", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,Logistic Regression,0.772532,0.803571,0.743802,0.772532
0,SVM,0.519313,0.519313,1.000000,0.683616
0,Decision tree,0.738197,0.720588,0.809917,0.762646


### 7 MLP

#### 7.1 Define parameter distribution

In [18]:
param_distributions_MLP = {
    'hidden_layer_sizes': [ (64,), (128,),(128,64), (64,128), (64,128,196), (196,128,64)],
    'activation': ['logistic', 'tanh', 'relu'],
    'solver': ['adam', 'sgd'],
    'alpha': [0, .0001, .0005, .001, .005],
    'batch_size': [25, 50, 100],
    'learning_rate': ['constant', 'invscaling', 'adaptive'],
    'learning_rate_init': [0.0005, 0.001, 0.005, 0.01],
    'max_iter': [5000],
    'tol': [0.000005, 0.00001, 0.00005],
    'early_stopping':[True],
    'n_iter_no_change':[5],
}

#### 7.2 Perform Random search

In [21]:
#Defining Scoring metric
score_measure = "recall"
kfolds = 5

#Create a MLP classifier
ann = MLPClassifier()

#Use RandomizedSearchCV to search over the parameter grid
random_search_MLP = RandomizedSearchCV(estimator = ann, param_distributions=param_distributions_MLP, cv=kfolds, n_iter=100,
                           scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)
# Fit the random search object to the data
_ = random_search_MLP.fit(X_train, y_train)

# Print the best parameters and score
bestRecallTree = random_search_MLP.best_estimator_
print(random_search_MLP.best_params_)




Fitting 5 folds for each of 100 candidates, totalling 500 fits
{'tol': 5e-06, 'solver': 'sgd', 'n_iter_no_change': 5, 'max_iter': 5000, 'learning_rate_init': 0.01, 'learning_rate': 'constant', 'hidden_layer_sizes': (196, 128, 64), 'early_stopping': True, 'batch_size': 50, 'alpha': 0.001, 'activation': 'logistic'}


In [77]:
y_pred = bestRecallTree.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       112
           1       0.52      1.00      0.68       121

    accuracy                           0.52       233
   macro avg       0.26      0.50      0.34       233
weighted avg       0.27      0.52      0.36       233



#### 7.3 Grid Search

In [24]:
#Define scoring measure
score_measure = "recall"
kfolds = 5

#Using best parameters found from random search
param_grid_MLPg = {
    'hidden_layer_sizes': [(196,128,64)],
    'activation': ['logistic'],
    'solver': ['sgd'],
    'alpha': [0.001],
    'learning_rate': ['constant'],
    'learning_rate_init': [0.01],
    'max_iter': [5000]
}

#Create a MLP classifier
ann = MLPClassifier()

# Create a grid search object
grid_search_MLP = GridSearchCV(estimator = ann, param_grid=param_grid_MLPg, cv=kfolds, 
                           scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)

# Fit the grid search to the data
_ = grid_search_MLP.fit(X_train, y_train)

# Print the best parameters and score
bestRecallTree = grid_search_MLP.best_estimator_

print(grid_search_MLP.best_params_)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
{'activation': 'logistic', 'alpha': 0.001, 'hidden_layer_sizes': (196, 128, 64), 'learning_rate': 'constant', 'learning_rate_init': 0.01, 'max_iter': 5000, 'solver': 'sgd'}


In [25]:
y_pred = bestRecallTree.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       112
           1       0.52      1.00      0.68       121

    accuracy                           0.52       233
   macro avg       0.26      0.50      0.34       233
weighted avg       0.27      0.52      0.36       233



In [27]:
c_matrix = confusion_matrix(y_test, grid_search_MLP.predict(X_test))
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"MLP", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,Logistic Regression,0.772532,0.803571,0.743802,0.772532
0,SVM,0.519313,0.519313,1.000000,0.683616
0,Decision tree,0.738197,0.720588,0.809917,0.762646
0,MLP,0.519313,0.519313,1.000000,0.683616


### 8. Keras techniques

In [29]:

# create model structure
def build_clf(meta, hidden_layer_sizes, dropout):
    n_features_in_ = meta["n_features_in_"]
    n_classes_ = meta["n_classes_"]
    target_encoder_ = meta["target_encoder_"]
    
    model = tf.keras.models.Sequential()
    model.add(keras.layers.Input(shape=n_features_in_)),
    #for hidden_layer_size in hidden_layer_sizes:
    for hidden_layer_size in hidden_layer_sizes:
        model.add(keras.layers.Dense(hidden_layer_size, 
            kernel_initializer= tf.keras.initializers.GlorotUniform(), 
            bias_initializer=keras.initializers.RandomNormal(mean=0.0, stddev=0.05, seed=None), 
            activation="relu"))
        model.add(keras.layers.Dropout(dropout))
    model.add(tf.keras.layers.Dense(10, activation='sigmoid'))
    
    

    return model






In [30]:
from scikeras.wrappers import KerasClassifier

keras_clf = KerasClassifier(
    model=build_clf,
    hidden_layer_sizes=64,
    dropout=0.5,
    optimizer=keras.optimizers.Adam,
    optimizer__learning_rate=0.0001
)
keras_clf.get_params()

{'model': <function __main__.build_clf(meta, hidden_layer_sizes, dropout)>,
 'build_fn': None,
 'warm_start': False,
 'random_state': None,
 'optimizer': keras.optimizers.optimizer_v2.adam.Adam,
 'loss': None,
 'metrics': None,
 'batch_size': None,
 'validation_batch_size': None,
 'verbose': 1,
 'callbacks': None,
 'validation_split': 0.0,
 'shuffle': True,
 'run_eagerly': False,
 'epochs': 1,
 'hidden_layer_sizes': 64,
 'dropout': 0.5,
 'optimizer__learning_rate': 0.0001,
 'class_weight': None}

#### 8.1 Define parameters

In [31]:
params = {
    
    
    'model__hidden_layer_sizes': [(70,),(90, ), (100,), (100, 90)], 
    'model__dropout': [0, 0.1],
    'batch_size':[20, 60, 100],
    'epochs':[10],
    'optimizer':['adam','sgd'],
    'loss':['sparse_categorical_crossentropy'],
    
    # this is added to the optimizer 
    'optimizer__learning_rate': [0.0001, 0.001, 0.01]

}
keras_clf.get_params()

{'model': <function __main__.build_clf(meta, hidden_layer_sizes, dropout)>,
 'build_fn': None,
 'warm_start': False,
 'random_state': None,
 'optimizer': keras.optimizers.optimizer_v2.adam.Adam,
 'loss': None,
 'metrics': None,
 'batch_size': None,
 'validation_batch_size': None,
 'verbose': 1,
 'callbacks': None,
 'validation_split': 0.0,
 'shuffle': True,
 'run_eagerly': False,
 'epochs': 1,
 'hidden_layer_sizes': 64,
 'dropout': 0.5,
 'optimizer__learning_rate': 0.0001,
 'class_weight': None}

#### 8.2 Perform Random Search

In [32]:
from sklearn.model_selection import RandomizedSearchCV

rnd_search_cv = RandomizedSearchCV(
    estimator=keras_clf, 
    param_distributions=params, 
    scoring='recall',  
    n_iter=50, 
    cv=3)

_ = rnd_search_cv.fit(X_train, y_train,  verbose=1)


Epoch 1/10
6/6 [==============================] - 1s 3ms/step - loss: 2.3085
Epoch 2/10
6/6 [==============================] - 0s 2ms/step - loss: 2.0631
Epoch 3/10
6/6 [==============================] - 0s 2ms/step - loss: 1.8377
Epoch 4/10
6/6 [==============================] - 0s 2ms/step - loss: 1.6262
Epoch 5/10
6/6 [==============================] - 0s 2ms/step - loss: 1.4322
Epoch 6/10
6/6 [==============================] - 0s 2ms/step - loss: 1.2601
Epoch 7/10
6/6 [==============================] - 0s 2ms/step - loss: 1.1073
Epoch 8/10
6/6 [==============================] - 0s 2ms/step - loss: 0.9756
Epoch 9/10
6/6 [==============================] - 0s 2ms/step - loss: 0.8670
Epoch 10/10
4/4 [==============================] - 0s 2ms/step
Epoch 1/10
7/7 [==============================] - 0s 2ms/step - loss: 2.4540
Epoch 2/10
7/7 [==============================] - 0s 2ms/step - loss: 2.1652
Epoch 3/10
7/7 [==============================] - 0s 2ms/step - loss: 1.9076
Epoch 4/10
7/

In [38]:
print(rnd_search_cv.best_params_)
best_model = rnd_search_cv.best_estimator_
print(best_model)


{'optimizer__learning_rate': 0.01, 'optimizer': 'adam', 'model__hidden_layer_sizes': (100, 90), 'model__dropout': 0, 'loss': 'sparse_categorical_crossentropy', 'epochs': 10, 'batch_size': 20}
KerasClassifier(
	model=<function build_clf at 0x000001FD61145900>
	build_fn=None
	warm_start=False
	random_state=None
	optimizer=adam
	loss=sparse_categorical_crossentropy
	metrics=None
	batch_size=20
	validation_batch_size=None
	verbose=1
	callbacks=None
	validation_split=0.0
	shuffle=True
	run_eagerly=False
	epochs=10
	hidden_layer_sizes=64
	dropout=0.5
	optimizer__learning_rate=0.01
	model__hidden_layer_sizes=(100, 90)
	model__dropout=0
	class_weight=None
)


In [39]:
c_matrix = confusion_matrix(y_test, rnd_search_cv.predict(X_test))
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"keras", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

12/12 [==============================] - 0s 1ms/step


,model,Accuracy,Precision,Recall,F1
0,Logistic Regression,0.772532,0.803571,0.743802,0.772532
0,SVM,0.519313,0.519313,1.000000,0.683616
0,Decision tree,0.738197,0.720588,0.809917,0.762646
0,MLP,0.519313,0.519313,1.000000,0.683616
0,keras,0.785408,0.798319,0.785124,0.791667


### Discussion of Result

The aim of this project was to develop a binary classifier that could predict whether an individual has heart disease or not. To achieve this, different modeling techniques, namely logistic regression, SVM, decision tree,MLP and keras technique were used. The primary goal was to select the model that minimized false negatives (i.e., a sick person being wrongly classified as healthy), as this had severe consequences. Additionally, high accuracy values were desirable, particularly true positives, given the evenly split data.

After comparing the five models, it was determined that the SVM model has the best recall score among all the models, indicating less false negative predictions, but the SVM model had a lower precision and accuracy score than the other models indicating less accurate identification of true positives. 

The MLP modelling techique also has a good recall score of 1.0000, accuracy of 0.519313, precision of 0.519313 and F1 score of 0.683616.The keras techique has a precision of 0.798319, the Accuracy is 0.785408, Recall of 0.785124 and F1 of 0.791667.

Both MLP and Keras modelling technique performed good.However, the decision tree model has a higher recall score of 0.809917, indicating fewer false negatives. It also has a good accuracy score of 0.738197 in this case high true positive gives indication of a good model.

Given the context of the data, model with high recall and accuracy should be selected, as the cost of false negatives is higher than that of false positives and it is also important to determine the true positives accurately(people with heart disease classified as sick). Decision tree would be the best fit model for this data, as it has a good recall score, as well as a good accuracy score.

###### 